Configuración del entorno necesario para:
* Cargar, explorar y procesar el dataset de calidad de vinos
* Preparar características usando VectorAssembler y StandardScaler.
* VectorAssembler sirve para combinar columnas de entrada en una sola columna de vectores.
* StandardScaler asegura que la normalización tenga una varianza estándar de 1.

In [ ]:
# Análisis y Modelo de Regresión Logística para Predecir la Calidad del Vino
# =========================================================================

# ## Introducción
# Este notebook está diseñado para cargar, explorar y preprocesar el dataset de calidad de vinos, y entrenar un modelo de regresión logística utilizando PySpark en un entorno de Databricks.

# ## 1. Configuración de Databricks y Bibliotecas
# Importamos las bibliotecas necesarias y configuramos la conexión con Databricks.

import sys
sys.executable

import findspark
findspark.init()

import pandas as pd
import pyspark

import pandas as pd
import numpy as np
#import pyspark.pandas as ps
from pyspark.sql import SparkSession
import matplotlib as plt
import seaborn as sns
from pyspark.sql.functions import col, sum



from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator


Crear e iniciar una sesión en Spark. Es la forma para usar PySpark.
Se inicia un entorno Spark local para procesar y analizar datos relacionados con la predicción de calidad de vinos. 

In [17]:

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('WineQualityPrediction')\
        .getOrCreate()

Cargar el csv.
* inferSchema = True: Le dice a PySpark que deduzca automáticamente el tipo de datos de cada columna basado en el contenido del archivo.
* sep=";": Especifica que el separador de columnas en el archivo es el punto y coma
* header = True: Indica que la primera fila del archivo contiene los nombres de las columnas

In [18]:
# ## 2. Carga de Datos
# Cargamos el archivo CSV con los datos del vino y revisamos las primeras filas.

# Cargar el archivo CSV

gt = spark.read.csv('data/winequality-red.csv', 
                       inferSchema = True,
                       sep=";", 
                       header = True)


In [19]:
print(type(gt))

<class 'pyspark.sql.dataframe.DataFrame'>


* Se asigna el DataFrame gt a una nueva variable llamada df. Esto no crea una copia, sino que ambas variables (gt y df) apuntan al mismo DataFrame.


In [20]:
df = gt

* La función withColumn se usa para crear o reemplazar una columna en el DataFrame. Aquí se crea una nueva columna llamada "label".
* col("quality") hace referencia a la columna "quality" en el DataFrame.
* col("quality") >= 6: Aplica una operación lógica: verifica si el valor de la columna "quality" es mayor o igual a 6.
Esto genera una columna temporal con valores booleanos: True (si la condición se cumple) o False (si no se cumple).
* .cast("int"): Convierte los valores booleanos de la columna temporal en enteros: True se convierte en 1. False se convierte en 0.
* df = ...:Reasigna el DataFrame df para incluir esta nueva columna "label".

El propósito es transformar la columna "quality" en una nueva columna llamada "label", que representa una clasificación binaria:
1 para vinos de calidad buena o excelente (calidad >= 6).
0 para vinos de calidad regular o baja (calidad < 6).

Esto prepara el dataset para ser usado en un modelo de clasificación, donde el objetivo será predecir la calidad del vino en dos clases (0 o 1).

In [ ]:
# Convertir la columna "quality" a una variable binaria en una columna llamada "label"

df = df.withColumn("label", (col("quality") >= 6).cast("int"))


* df.columns: Devuelve una lista con los nombres de todas las columnas del DataFrame df.
* [c for c in df.columns]:Es una comprensión de lista que itera sobre cada columna c en df.columns.
* if c not in ["quality", "label"]: Incluye solo aquellas columnas cuyo nombre no está en la lista ["quality", "label"].
* Esto excluye específicamente:
"quality": La columna original que contiene las calificaciones de calidad de los vinos.
"label": La columna creada anteriormente para la clasificación binaria.
* feature_columns = ...: Asigna la lista resultante a la variable feature_columns.

El propósito es preparar los datos para el modelo. Se seleccionan las columnas relevantes como características, excluyendo:
* "quality": Ya que fue utilizada para derivar la columna "label".
* "label": Porque esta es la variable objetivo (target) del modelo.

In [ ]:
# Selección de características

feature_columns = [c for c in df.columns if c not in ["quality", "label"]]


VectorAssembler:
* Es una clase de pyspark.ml.feature que combina múltiples columnas (de tipo numérico) en un único vector. Este vector es necesario para muchos algoritmos de Machine Learning en PySpark.

inputCols=feature_columns:
* Define las columnas de entrada que se combinarán en el vector.
En este caso, feature_columns es la lista de nombres de columnas creada previamente, que contiene las características (excluyendo "quality" y "label").

outputCol="features":

* Especifica el nombre de la nueva columna donde se almacenará el vector de características. Aquí, se llamará "features".

assembler.transform(df):
* Aplica la transformación al DataFrame df. Esto genera una nueva columna llamada "features" que contiene el vector combinado para cada fila.

df = ...:
* Reasigna el DataFrame df con la nueva columna "features" incluida.

Así, el DataFrame df ahora tiene una nueva columna llamada "features", que contiene un vector denso con los valores de todas las columnas de características para cada fila.

In [ ]:
# Ensamblaje de las características en un solo vector

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df)

Este fragmento de código utiliza la clase StandardScaler de PySpark para escalar (normalizar) las características del DataFrame. La normalización es importante en muchos modelos de Machine Learning porque garantiza que las características estén en la misma escala.

StandardScaler:
* Es una clase de pyspark.ml.feature que escala las características numéricas para que tengan:
Media = 0 (si se usa centering, no activado por defecto).
Varianza = 1.
* Este proceso es útil para algoritmos que son sensibles a la magnitud de las características (por ejemplo, regresión logística o SVM).

inputCol="features":
* Especifica la columna de entrada que contiene los vectores de características. En este caso, es la columna "features", creada anteriormente con VectorAssembler.

outputCol="scaledFeatures":
* Especifica el nombre de la nueva columna que contendrá las características escaladas. En este caso, se llamará "scaledFeatures".

scaler.fit(df):
* Ajusta el escalador (StandardScaler) a los datos del DataFrame df. Esto calcula:
- La media (si se activa el centrado).
- La desviación estándar de las columnas en el vector de características.
* Devuelve un modelo ajustado (scaler_model) que puede usarse para transformar los datos.

scaler_model.transform(df):

* Aplica el modelo ajustado para escalar las características en la columna "features".
* Genera una nueva columna, "scaledFeatures", que contiene las características escaladas.

df = ...:

* Reasigna el DataFrame df con la nueva columna "scaledFeatures" incluida.

El resultado es que el DataFrame df ahora incluye una nueva columna "scaledFeatures", que contiene los vectores escalados.
Cada característica en "scaledFeatures" está normalizada con una desviación estándar de 1.

Este paso asegura que todas las características tengan la misma importancia relativa en el modelo, evitando que las características con valores grandes dominen el aprendizaje.

In [ ]:
# Estandarización de las características

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scaler_model = scaler.fit(df)
df = scaler_model.transform(df)

df.randomSplit([0.8, 0.2], seed=42):

* randomSplit: Método de PySpark que divide un DataFrame en varios subconjuntos de manera aleatoria.

* [0.8, 0.2]: Define las proporciones de la división. Aquí:

- El 80% de los datos se asigna al conjunto de entrenamiento (train_data).
- El 20% de los datos se asigna al conjunto de prueba (test_data).
seed=42: Especifica una semilla para el generador de números aleatorios. Esto garantiza que la división sea reproducible: cada vez que se ejecuta el código con la misma semilla, los subconjuntos serán iguales.

train_data, test_data = ...:

* Almacena los subconjuntos generados en las variables:
- train_data: Contiene el 80% de los datos, utilizado para entrenar el modelo.
- test_data: Contiene el 20% de los datos, utilizado para evaluar el modelo.

La división en conjuntos de entrenamiento y prueba es crucial para evitar overfitting y evaluar el desempeño del modelo de manera objetiva. Esto permite:

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

LogisticRegression:
* Es una clase de pyspark.ml.classification que implementa el modelo de regresión logística.
* La regresión logística es un modelo de clasificación lineal que predice la probabilidad de pertenecer a una clase 

featuresCol="scaledFeatures":

* Especifica que la columna "scaledFeatures" contiene los vectores de características que se utilizarán como entrada para el modelo.
* Esta columna fue creada previamente al escalar las características originales.

labelCol="label":

* Especifica que la columna "label" contiene las etiquetas de la variable objetivo.
* Estas etiquetas son 0 o 1, creadas anteriormente a partir de la columna "quality".

lr.fit(train_data):

* Ajusta el modelo de regresión logística (lr) a los datos de entrenamiento (train_data).
* Esto implica que el modelo aprende los pesos (coeficientes) para cada característica en "scaledFeatures" que mejor predicen la variable objetivo "label".

lr_model:

* Contiene el modelo entrenado. Este objeto almacena:
Los coeficientes y el intercepto del modelo.
Información sobre el rendimiento del ajuste en los datos de entrenamiento.

In [26]:
# Entrenar el modelo de regresión logística
lr = LogisticRegression(featuresCol="scaledFeatures", labelCol="label")
lr_model = lr.fit(train_data)

lr_model.transform(test_data):

* Aplica el modelo entrenado (lr_model) al conjunto de datos de prueba (test_data).
* Este proceso genera un nuevo DataFrame que incluye:
- Las columnas originales del conjunto de prueba.
- Nuevas columnas con los resultados de las predicciones.

predictions:

* Es el DataFrame resultante que contiene las predicciones realizadas por el modelo.

predictions: Es un DataFrame que contiene tanto los datos originales como las predicciones y probabilidades generadas por el modelo para cada instancia en el conjunto de prueba.

In [27]:
# ## 6. Evaluación del Modelo

# Realizar predicciones en el conjunto de prueba
predictions = lr_model.transform(test_data)


Este fragmento de código utiliza el evaluador BinaryClassificationEvaluator de PySpark para calcular el área bajo la curva ROC (AUC-ROC) de las predicciones realizadas por el modelo. La AUC-ROC mide la capacidad del modelo para distinguir entre las clases en un problema de clasificación binaria. 

BinaryClassificationEvaluator:

* Es una clase de PySpark que evalúa el rendimiento de un modelo de clasificación binaria.
* Calcula métricas basadas en las predicciones generadas.

labelCol="label":

* Especifica que la columna "label" contiene las etiquetas verdaderas (0 o 1) de las clases.

rawPredictionCol="rawPrediction":

* Especifica que la columna "rawPrediction" contiene los valores generados por el modelo antes de aplicar la función sigmoide o softmax.
* Esta columna se utiliza para calcular métricas como el área bajo la curva ROC.

metricName="areaUnderROC":

* Indica que el evaluador debe calcular la métrica de área bajo la curva ROC (AUC-ROC).

evaluator.evaluate(predictions):

* Calcula el valor de la métrica especificada (areaUnderROC) usando el DataFrame predictions, que contiene tanto las etiquetas verdaderas como las predicciones del modelo.

roc_auc:

* Contiene el valor calculado de AUC-ROC

In [28]:
# Evaluador para medir el área bajo la curva ROC
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
roc_auc = evaluator.evaluate(predictions)
print(f"Área bajo la curva ROC: {roc_auc:.2f}")

Área bajo la curva ROC: 0.84


Este fragmento de código utiliza el evaluador MulticlassClassificationEvaluator para calcular la exactitud (accuracy) del modelo en sus predicciones. La exactitud mide la proporción de predicciones correctas frente al total de predicciones realizadas. 

MulticlassClassificationEvaluator:

* Es una clase de PySpark que evalúa el rendimiento de modelos de clasificación, ya sea binaria o multiclase.
* Aunque en este caso el modelo es binario (0 o 1), este evaluador funciona igual de bien para clasificación binaria.

labelCol="label":

* Especifica que la columna "label" contiene las etiquetas reales de las clases (0 o 1).

predictionCol="prediction":

* Indica que la columna "prediction" contiene las predicciones del modelo para cada fila (0 o 1 en este caso).

metricName="accuracy": 

* Especifica que se debe calcular la métrica de exactitud: Predicciones correctas / total de predicciones 



In [ ]:
# Calcular la precisión y mostrar el informe de clasificación
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_accuracy.evaluate(predictions)
print(f"Exactitud del modelo: {accuracy:.2f}")

Exactitud del modelo: 0.77


In [30]:
# Mostrar una muestra de las predicciones
predictions.select("label", "prediction", "probability").show(5)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|    1|       1.0|[0.01812103566953...|
|    1|       1.0|[0.25500976345328...|
|    0|       0.0|[0.79508949677236...|
|    0|       0.0|[0.76399102429231...|
|    1|       1.0|[0.04814519001736...|
+-----+----------+--------------------+
only showing top 5 rows

